In [2]:
import torch
from torchvision import datasets, transforms as T


import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import time
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import argparse


In [11]:

from tensorboardX import SummaryWriter

import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

from torchvision import datasets


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [9]:
!pip install dataset

     |████████████████████████████████| 210 kB 45.1 MB/s 
     |████████████████████████████████| 78 kB 7.7 MB/s 


In [12]:
#start the time
t = time.time()


In [17]:
def fit():
    # Create batches for test data
    test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)
    train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug,num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)
    #summary_writer = tf.summary.create_file_writer('./log/{}'.format(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")))
    train_summary_writer = tf.summary.create_file_writer('./logs/train')
    test_summary_writer = tf.summary.create_file_writer('./logs/test')    

    for epoch in range(EPOCHS):
        train_loss = test_loss = train_acc = test_acc = 0.0
        with train_summary_writer.as_default():
            tf.summary.trace_on(graph=True, profiler=True) # To start capturing profiler in Tensorboard
            tf.keras.backend.set_learning_phase(1)
            for (x, y) in tqdm(train_set):# Iterate over the batches of train dataset objects .
                with tf.GradientTape() as tape:
                    #logits,actual  value for this minibatch
                    loss, correct = model(x, y)
                    var = model.trainable_variables
                    grads = tape.gradient(loss, var)
                    opt.apply_gradients(zip(grads, var))
                    global_step.assign_add(1)
                # Add extra losses created during this forward pass
                train_loss += loss.numpy()
                train_acc += correct.numpy()

            tf.summary.scalar('train loss', train_loss/len_train, step=epoch)
            tf.summary.scalar('train acc', train_acc/len_train, step=epoch)
            tf.summary.trace_export(name="train_trace", step=epoch)

        with test_summary_writer.as_default():
            tf.summary.trace_on(graph=True, profiler=True)
            tf.keras.backend.set_learning_phase(0)
            for (x, y) in tqdm(test_set):
                loss, correct = model(x, y)
                test_loss += loss.numpy()
                test_acc += correct.numpy()
            tf.summary.scalar('test loss', test_loss/len_test, step=epoch)
            tf.summary.scalar('test acc', test_acc/len_test, step=epoch)
            tf.summary.trace_export(name="test_trace", step=epoch)
            print('Epoch: {}/{}'.format(epoch+1, EPOCHS),
              'Train Loss: {:.4f}'.format(train_loss/len_train),
              'Train Acc: {:.4f}'.format(train_acc/len_train),
              'Test Loss: {:.4f}'.format(test_loss/len_test),
              'Test Acc: {:.4f}'.format(test_acc/len_test))
        # Save the model every 5 epochs
        if (epoch + 1) % 5 == 0:
            model.save_weights('./checkpoints/ckpt_{}'.format(epoch+1))
                #end the time
    print('Time taken for training: {} seconds'.format(time.time()-t))